In [69]:
# states are
# 0 - running
# 1 - waiting for input
# 99 - halt

mutable struct Prog
    code::Vector{Int}
    cur::Int
    input::Vector{Int}
    output::Vector{Int}
    relative_base::Int
    state::Int
end

str2prog(s) = parse.(Int, split(s, ","))

Prog(s::String) = Prog(vcat(str2prog(s), zeros(Int, 10000)), 1, [], [], 0, 0)

struct Instruction
    op::Int
    modes::Vector{Int}
end

function Instruction(op_code::Int)
    ops = Dict{Int, Int}(
        1 => 3,
        2 => 3,
        3 => 1,
        4 => 1,
        5 => 2,
        6 => 2,
        7 => 3,
        8 => 3,
        9 => 1,
        99 => 0
    )
    
    op = mod(op_code, 100)
    modes_code = div(op_code, 100)
    modes = zeros(ops[op])
    for i in 1:ops[op]
        modes[i] = mod(modes_code, 10)
        modes_code = div(modes_code, 10)
    end
    
    Instruction(op, modes)
end

Instruction(prog::Prog) = Instruction(prog.code[prog.cur])

function take(prog::Prog, mode, offset)
    if mode == 0
        return prog.code[prog.code[prog.cur + offset] + 1]
    elseif mode == 1
        return prog.code[prog.cur + offset]
    else
        return prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1]
    end
end

function update!(prog::Prog, value, offset, mode = 0)
    if mode == 0
        prog.code[prog.code[prog.cur + offset] + 1] = value
    elseif mode == 2
        prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1] = value
    end
end

function apply(prog::Prog, instruction::Instruction)
    #println(instruction)
    if instruction.op == 99
        prog.state = 99
    elseif instruction.op == 1
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 + a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 2
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 * a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 3
        if isempty(prog.input)
            prog.state = 1
        else
            update!(prog, popfirst!(prog.input), 1, instruction.modes[1])
            prog.cur += 2
        end
    elseif instruction.op == 4
        push!(prog.output, take(prog, instruction.modes[1], 1))
        prog.cur += 2
    elseif instruction.op == 5
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 != 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 6
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 == 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 7
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 < a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 8
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 == a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 9
        prog.relative_base += take(prog, instruction.modes[1], 1)
        prog.cur += 2
    end
end

function run(prog::Prog, input::Vector{Int})
    prog.input = input
    run(prog)
end

function run(prog)
    while prog.state == 0
        instruction = Instruction(prog)
        apply(prog, instruction)
    end
    
    prog.output
end

function feed(prog::Prog, input::Vector{Int})
    prog.input = vcat(prog.input, input)
    prog.state = 0
end

clear(prog) = (prog.output = [])

clear (generic function with 1 method)

In [112]:
struct Robot
    pos::Vector{Int}
    dir::Vector{Int}
end

Robot() = Robot([0, 0], [-1, 0])

rotate(r::Robot, dir::Int) = Robot(r.pos, (dir == 0 ? -1 : 1) * [0 1; -1 0] * r.dir)
shift(r::Robot) = Robot(r.pos + r.dir, r.dir)

shift (generic function with 1 method)

In [37]:
r = Robot()
r = rotate(r, 0)
println(r)
r = rotate(r, 0)
println(r)
r = rotate(r, 0)
println(r)
r = rotate(r, 1)
println(r)
r = rotate(r, 1)
println(r)
r = rotate(r, 1)
println(r)

Robot([0, 0], [0, -1])
Robot([0, 0], [-1, 0])
Robot([0, 0], [0, 1])
Robot([0, 0], [-1, 0])
Robot([0, 0], [0, -1])
Robot([0, 0], [1, 0])


In [58]:
prog = Prog(readline("input.txt"))

Prog([3, 8, 1005, 8, 338, 1106, 0, 11, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1, Int64[], Int64[], 0, 0)

In [77]:
prog = Prog(readline("input.txt"))
board = Dict{Vector{Int}, Int}()
r = Robot()
while true
    current_color = get(board, r.pos, 0)
    feed(prog, [current_color])
    res = run(prog)
    clear(prog)
    if prog.state == 99 break end
    board[r.pos] = res[1]
    r = rotate(r, res[2])
    r = shift(r)
end

In [78]:
board

Dict{Array{Int64,1},Int64} with 2720 entries:
  [11, -37] => 1
  [11, 6]   => 1
  [31, -80] => 1
  [17, -59] => 1
  [51, -52] => 1
  [10, -53] => 1
  [-4, 0]   => 1
  [4, 8]    => 0
  [-2, 1]   => 1
  [19, -6]  => 0
  [37, -75] => 1
  [30, -55] => 1
  [30, -79] => 1
  [18, -70] => 0
  [44, -51] => 1
  [53, -41] => 0
  [4, -53]  => 0
  [12, -23] => 1
  [53, -47] => 1
  [53, -42] => 1
  [15, -38] => 0
  [12, -9]  => 0
  [19, -22] => 1
  [15, -60] => 0
  [16, -6]  => 0
  ⋮         => ⋮

## Part 2

In [119]:
prog = Prog(readline("input.txt"))
board = Dict{Vector{Int}, Int}()
board[[0, 0]] = 1
r = Robot()
while true
    current_color = get(board, r.pos, 0)
    feed(prog, [current_color])
    res = run(prog)
    clear(prog)
    if prog.state == 99 break end
    board[r.pos] = res[1]
    r = rotate(r, res[2])
    r = shift(r)
end

In [120]:
dots = sort(collect(keys(board)))
x1 = minimum([d[1] for d in dots])
y1 = minimum([d[2] for d in dots])
x2 = maximum([d[1] for d in dots])
y2 = maximum([d[2] for d in dots])

pic = zeros(Int, x2 - x1 + 1, y2 - y1 + 1)
for d in dots
    pic[d[1] - x1 + 1, d[2] - y1 + 1] = board[d]
end

In [121]:
pic2 = map(x -> x == 1 ? '⬛' : '⬜', pic)
for i in 1:size(pic2)[1]
    println(join(pic2[i, :]))
end

⬜⬜⬜⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬛⬜⬜⬜⬜⬛⬛⬜⬛⬛⬛⬜⬜⬜⬛⬛⬜⬜⬜⬛⬛⬜⬜⬜⬜⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜⬜⬛⬜⬛⬜⬜⬛⬜⬜⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜⬛⬜⬜⬛⬜⬜⬛⬜⬜⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬜⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬛⬜⬜⬜⬛⬛⬛⬜⬜⬜⬜⬜⬛⬜⬛⬛⬛⬜⬜⬛⬛⬛⬛⬜⬛⬜⬛⬛⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬛⬜⬜⬛⬜⬛⬜⬜⬜⬜⬛⬜⬜⬜⬜⬛⬜⬜⬛⬜⬛⬜⬛⬜⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬜⬜
⬜⬜⬛⬛⬜⬜⬛⬛⬛⬛⬜⬛⬜⬜⬜⬜⬜⬛⬛⬜⬜⬛⬜⬜⬛⬜⬛⬜⬜⬛⬜⬜⬛⬛⬛⬜⬜⬛⬛⬜⬜⬜⬜


In [62]:
run(prog, [0])
clear(prog)
feed(prog, [0])
run(prog, [0])

2-element Array{Int64,1}:
 1
 0

In [4]:
prog.state

1

In [10]:
rot = [0 1; -1 0]

2×2 Array{Int64,2}:
  0  1
 -1  0

In [14]:
rot * rot * rot * rot * [1, 0]

2-element Array{Int64,1}:
 1
 0

In [16]:
-1 * [0 1; -1 0]

2×2 Array{Int64,2}:
 0  -1
 1   0

MethodError: MethodError: no method matching *(::Array{Int64,2}, ::Tuple{Int64,Int64})
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  *(::Union{DenseArray{TX,2}, Base.ReinterpretArray{TX,2,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray}, Base.ReshapedArray{TX,2,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:Union{Base.ReinterpretArray{T,N,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray} where N where T, SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray}, SubArray{TX,2,A,I,L